In [ ]:
## site de emploi.cm

In [ ]:
from bs4 import BeautifulSoup as bs
import requests as rq
import pandas as pd

In [ ]:
# verifier la connection a la page
url = "https://www.emploi.cm/recherche-jobs-cameroun/"
reponse = rq.get(url)
print(reponse.status_code)

# ouvrir le site (contenu HTML) avec Beautifulsoup
soup = bs(reponse.content,"html.parser")
#soup

200


In [ ]:
# récuperer les liens de chaque page
urls = []
for i in range(1,11):
    i = f"https://www.emploi.cm/recherche-jobs-cameroun?page-{i}.html"
    urls.append(i)
urls

['https://www.emploi.cm/recherche-jobs-cameroun?page-1.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-2.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-3.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-4.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-5.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-6.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-7.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-8.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-9.html',
 'https://www.emploi.cm/recherche-jobs-cameroun?page-10.html']

In [ ]:
#  extraire les catégories et leurs URL

     # Stocker les noms de catégories et les liens
# récuperer les liens de chaque page
urls = []
category_names = []

category_list = soup.find('div', class_='footer-middle-item accordeon mobile-only').find_all('a')

for i in range(1,11):
    i = f"https://www.emploi.cm/recherche-jobs-cameroun?page-{i}.html"
    urls.append(i)

    for category in category_list:
      category_names.append(category.text.strip())

print(category_names)


['Achats Cameroun', 'Commercial, vente Cameroun', 'Gestion, comptabilité, finance Cameroun', 'Informatique, nouvelles technologies Cameroun', 'Juridique Cameroun', 'Management, direction générale Cameroun', 'Marketing, communication Cameroun', 'Métiers de la santé et du social Cameroun', 'Métiers des services Cameroun', 'Métiers du BTP Cameroun', 'Production, maintenance, qualité Cameroun', 'R&D, gestion de projets Cameroun', 'RH, formation Cameroun', 'Secrétariat, assistanat Cameroun', 'Télémarketing, téléassistance Cameroun', 'Tourisme, hôtellerie, restauration Cameroun', 'Transport, logistique Cameroun', 'Achats Cameroun', 'Commercial, vente Cameroun', 'Gestion, comptabilité, finance Cameroun', 'Informatique, nouvelles technologies Cameroun', 'Juridique Cameroun', 'Management, direction générale Cameroun', 'Marketing, communication Cameroun', 'Métiers de la santé et du social Cameroun', 'Métiers des services Cameroun', 'Métiers du BTP Cameroun', 'Production, maintenance, qualité Cam

In [ ]:
liens = []
for url in urls:
    lien_offre = rq.get(url)
    offre_soup = bs(lien_offre.text, 'html.parser')
    lien = offre_soup.find_all('div', class_='card-job-detail')
    for l in lien:
        lien = l.find('a').get('href')
        liens.append(lien)


In [ ]:
len(liens)

250

In [43]:
#définition des listes
titre_offres=[]
description_offres = []
date_publication = []
nom_entreprise = []
type_contrat = []
niveau_etude = []
niveau_experience = []
localisation = []
competence = []
categorie_offre = []
# boucle pour parcourrir toutes les pages
for url, category in zip(urls, category_names):
  lien_page = rq.get(url)
  page_soup = bs(lien_page.text, 'html.parser')
  # pacourrir chaque offre
  offres = page_soup.find_all('div', class_='card-job-detail')
  for offre in offres:
    #titre de l'offre
    titre = offre.find('h3').get_text().strip()
    titre_offres.append(titre)
    # description de l'offre
    description = offre.find('p').get_text().strip()
    description_offres.append(description)
    # date de publication
    date = offre.find('time').get_text().strip()
    date_publication.append(date)
    # nom de l'entreprise
    nom = offre.find('a', class_='card-job-company company-name').get_text().strip()
    nom_entreprise.append(nom)
    # categorie
    categorie_offre.append(category)
    ul_element = offre.find('ul') # Find the first 'ul' element within each 'offre'
    if ul_element:
        list_items = ul_element.find_all('li')

        #type de contrat
        if len(list_items) > 2:
            Type = ''.join(list_items[2].get_text().replace("Contrat proposé\n", "")).strip()
            type_contrat.append(Type)

            #niveau d'étude
        if len(list_items) > 0:
            etude = list_items[0].get_text().strip().replace("Niveau d´études requis :", "")
            niveau_etude.append(etude)

            #niveau d'expérience
        if len(list_items) > 1:
            experience = list_items[1].get_text().strip().replace("Niveau d'expérience :", "")
            niveau_experience.append(experience)

            #localisation
        if len(list_items) > 3:
            local = list_items[3].get_text().strip().replace("Région de :", "")
            localisation.append(local)

            #competence
        if len(list_items) > 4:
            competences = list_items[4].get_text().strip().replace("Compétences clés :", "")
            competence.append(competences)

In [44]:

# ceation du dictionnaire avec mes liste ce qui permettra de creer le dataframe
emploicm_scrap = {
    "nom_entreprise": nom_entreprise,
    "titre_offres": titre_offres,
    "description_offres": description_offres,
    "date_publication": date_publication,
    "localisation":localisation,
    "type_contrat": type_contrat,
    "niveau_etude":niveau_etude,
    "niveau_experience":niveau_experience,
    "competence":competences,
    "categorie_offre":categorie_offre
    }

In [45]:
emploicm_scrap =pd.DataFrame(emploicm_scrap)

In [49]:
emploicm_scrap

,nom_entreprise,titre_offres,description_offres,date_publication,localisation,type_contrat,niveau_etude,niveau_experience,competence,categorie_offre
0,OPTIMUM INTERIM,Créateur/créatrice de Blogs - Télétravail,Poste proposé : Créateur/créatrice de Blogs ...,21.06.2024,Bafoussam - Bamenda - Bertoua - Buéa - Douala...,: Freelance,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 - Bac+5 et plus",Débutant < 2 ans et plus,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Achats Cameroun
1,YANOU I HOLDINGS SARL,Stagiaire Assistant Technique/Commercial - Douala,Ses responsabilités sont comme suit : Être p...,21.06.2024,Douala,: Stage,Bac+2,"Etudiant, jeune diplômé",DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Achats Cameroun
2,ETUIDER EN FRANCE - ETEF,Business Manager & Responsable d´Agence - Douala,Nous sommes à la recherche d'un Business Mana...,21.06.2024,Bafoussam - Bamenda - Bertoua - Buéa - Douala...,: CDD,"Bac+3, Bac+4 - Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Achats Cameroun
3,L'HÔTEL LA VALLEE DE BANA,DJ - Bana,Le DJ a pour mission d’animer les soirées du...,21.06.2024,Bafoussam,": CDI, CDD & Stage",Bac,Expérience entre 2 ans et 5 ans,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Achats Cameroun
4,KEMETHOME,Commercial - Yaoundé,Mission : Sous la supervision d'un responsa...,21.06.2024,Bafoussam - Bamenda - Bertoua - Buéa - Douala...,: CDI,"Bac+3, Bac+4 - Bac+5 et plus","Etudiant, jeune diplômé et plus",DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Achats Cameroun
...,...,...,...,...,...,...,...,...,...,...
245,ADVENT SOFT,Stagiaire Support Technique Applicatif - Yaoundé,La société Advent Soft Sarl recrute pour ses b...,19.06.2024,Yaoundé,: Stage,Bac+2,Débutant < 2 ans,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Métiers du BTP Cameroun
246,ADVENT SOFT,Développeur Full Stack Java/Spring/Angular,La société ADVENT SOFT recherche pour ses be...,19.06.2024,Yaoundé,: CDD,Bac+3,Expérience entre 2 ans et 5 ans,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Métiers du BTP Cameroun
247,AMARAL CONSULTING,Commercial - Douala,Nous sommes à la recherche de Commerciaux. ...,18.06.2024,Bafoussam - Bamenda - Bertoua - Buéa - Douala...,: Freelance - Temps partiel,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 - Bac+5 et plus","Etudiant, jeune diplômé et plus",DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Métiers du BTP Cameroun
248,RODEC CAMEROUN,QA Engineer,"You are : A detail oriented, organized QA to...",18.06.2024,Bafoussam - Bamenda - Bertoua - Buéa - Douala...,: CDI,Bac+3,Expérience entre 2 ans et 5 ans,DIGITAL MARKETING - MARKETING - NEWSLETTERS -...,Métiers du BTP Cameroun


In [48]:
emploicm_scrap.to_csv("emploicm_scrap.csv")